<h3>Librerías, paquetes y funciones usadas en el cuaderno</h3>

In [3]:
import cv2
import numpy as np

<h3>Función para la obtención del color del parqué y de la zona</h3>

In [4]:
# Función para manejar los clics del ratón
def get_color(event, x, y, flags, param):
    global selected_colors
    if event == cv2.EVENT_LBUTTONDOWN:
        selected_color = frame[y, x]  # Obtener el color en el punto clicado
        selected_colors.append(selected_color)
        print(f"Color seleccionado (BGR): {selected_color}")
        if len(selected_colors) == 2:  # Si ya se seleccionaron dos colores, cerrar la ventana
            cv2.destroyWindow("Parquet y zona.")

<h3>Selección de colores del parqué y de la zona para generar una máscara</h3>

In [ ]:
# Leer el video
video_path = "./assets/videos/dal-lac1.mp4"
cap = cv2.VideoCapture(video_path)

# Variables globales para almacenar los colores seleccionados
selected_colors = []

ret, frame = cap.read()  # Leer el primer fotograma
if not ret:
    print("Error al leer el video.")
    cap.release()
    exit()

# Mostrar el primer fotograma para seleccionar colores
cv2.imshow("Parquet y zona.", frame)
cv2.setMouseCallback("Parquet y zona.", get_color)

print("1. Haz clic en el parquet para seleccionar un color.")
print("2. Haz clic en la zona bajo la canasta para seleccionar otro color.")


# Esperar hasta que el usuario seleccione dos colores
while len(selected_colors) < 2:
    cv2.waitKey(1)

# Convertir los colores seleccionados a espacio HSV
selected_colors_hsv = [cv2.cvtColor(np.uint8([[color]]), cv2.COLOR_BGR2HSV)[0][0] for color in selected_colors]

# Crear máscaras para cada color seleccionado
masks = []
for color_hsv in selected_colors_hsv:
    lower_bound_hsv = np.clip(color_hsv - [10, 50, 50], 0, 255)
    upper_bound_hsv = np.clip(color_hsv + [10, 50, 50], 0, 255)

    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gaussian_hsv = cv2.GaussianBlur(frame_hsv, (5,5), 0)
    #gaussian_hsv = cv2.bilateralFilter(frame_hsv, 6, 30, 30)

    mask = cv2.inRange(gaussian_hsv, lower_bound_hsv, upper_bound_hsv)
    masks.append(mask)

# Combinar las máscaras
combined_mask = cv2.bitwise_or(masks[0], masks[1])

# Invertir la máscara combinada y aplicar para eliminar las áreas seleccionadas
mask_inv = cv2.bitwise_not(combined_mask)
masked_frame = cv2.bitwise_and(frame, frame, mask=mask_inv)

# Mostrar los resultados
cv2.imshow("Original", frame)
cv2.imshow("Parquet y zona eliminadas", masked_frame)

cv2.waitKey(0)
cv2.destroyAllWindows()
cap.release()


In [ ]:
import numpy as np
import cv2
from skimage import transform

# Captura cuatro clics del ratón (parte uno)
def get_points(event, x, y, flags, param):
    puntos = param["points"]
    img = param["image"].copy()

    if event == cv2.EVENT_LBUTTONDOWN:  # Botón izquierdo
        puntos.append((x, y))  
        
        # Dibujar en la imagen
        cv2.circle(img, (x, y), 5, (0, 255, 0), -1)
        cv2.imshow(param["wname"], img)

        # Cuarto puntos, condición de parada
        if len(puntos) == 4:
            cv2.destroyAllWindows()
            print("Coordenadas imagen:", puntos)

# Movimiento del puntero (parte 2)
def mouse_event(event, x, y, flags, param):
    homo = param["tform"]
    img = param["image"]

    if event == cv2.EVENT_MOUSEMOVE:  
        # Transformar posición del ratón a imagen destino
        print("uwu")
        punto_A = np.array([[x, y]])  # Puntero 
        punto_B = homo(punto_A)  # Transformar a imagen destino

        # Dibuja en la imagen destino
        print("wuwu")
        imgtmp = img.copy()
        cv2.circle(imgtmp, (int(punto_B[0][0]), int(punto_B[0][1])), 5, (0, 255, 0), -1)
        cv2.imshow(param["wname"], imgtmp)

############## 1. RECOPILA LOS CUATRO PUNTOS EN AMBAS IMÁGENES
#Inicializa listas depuntos
puntosA = []
puntosB = []

video_path = "./assets/videos/okc-nyk.mp4"
cap = cv2.VideoCapture(video_path)

# Variables globales para almacenar los colores seleccionados
selected_colors = []

ret, frame = cap.read()  # Leer el primer fotograma
if not ret:
    print("Error al leer el video.")
    cap.release()
    exit()

#Lee imágenes, crea copias de trabajo
diagrama = cv2.imread('./assets/court_diagrams/black_court.png')

frametmp = frame.copy()

# aqui toco cosas
frametmp = masked_frame

diagramatmp = diagrama.copy()

#Vista del campo
cv2.imshow("Vista", frametmp) 
params = {
    "points": puntosA,
    "image": frametmp, 
    "wname": "Vista"
}
cv2.setMouseCallback("Vista", get_points, params)

# Selecciona cuatro puntos o cierra ventana
cv2.waitKey(0)

#Vista ceintal
cv2.imshow("Mapa", diagrama) 
params = {
    "points": puntosB,
    "image": diagramatmp, 
    "wname": "Mapa"
}
cv2.setMouseCallback("Mapa", get_points, params)

# Selecciona cuatro puntos o cierra ventana
cv2.waitKey(0)
cv2.destroyAllWindows()

print("hola")
print("Coordenadas campo:", puntosA)
print("Coordenadas diagrama:", puntosB)
#Transformación de los puntos https://scikit-image.org/docs/stable/auto_examples/transform/plot_transform_types.html
tform = transform.estimate_transform('projective', np.array(puntosA), np.array(puntosB))
print("adios")

###### 2. MUESTRA POSICIÓN DEL RATÓN EN EL CAMPO, EN LA IMAGEN DEL MAPA
# Mostrar las dos imágenes
cv2.imshow("Vista", frame)
cv2.imshow("Mapa", diagrama)

params = {
    "tform": tform,
    "image": diagrama, 
    "wname": "Mapa"
}

# Evento de movimiento del ratón
cv2.setMouseCallback("Vista", mouse_event, params)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
import cv2
import numpy as np
from skimage import transform
from ultralytics import YOLO  #Necesario para detectar con YOLO

#Intersección enter dos líneas
def line_intersection(line1, line2):
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2

    # Calcular  pendientes e intersecciones
    denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    if denom == 0:  # Líneas paralelas o coincidentes
        return None

    intersect_x = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denom
    intersect_y = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denom

    return (intersect_x, intersect_y)

#Comprueba que el punto esté dentro de la imagen
def is_within_image(point, image_width, image_height):
    x, y = point
    return 0 <= x < image_width and 0 <= y < image_height

#Carga modelo YOLO
detect_model = YOLO('./yolo_models/yolo11n.pt') #Contenedores
segment_model = YOLO("./yolo_models/segment.pt")
# Etiqueta de las clases o clase de interés
classNames = ["person"]

# Carga imagen
image = masked_frame
height, width, _ = image.shape  
# Convierte a HSV
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
# Muestra entrada
cv2.imshow("Input", image)

# Rango de verde de interés en HSV
# Se selecciónan los valores minimos y máximos con los del parquet y zona

# Máscara de zona verde
mask = mask_inv
# Muestra máscara
cv2.imshow("Máscara", mask)

masked_frame_gray = cv2.cvtColor(masked_frame, cv2.COLOR_BGR2GRAY)
canny_masked_frame = cv2.Canny(masked_frame_gray, 50, 150)

cv2.imshow("canny", canny_masked_frame)

# Convierte a grises y detecta de bordes
# gray = cv2.cvtColor(canny_masked_frame, cv2.COLOR_BGR2GRAY)

# Detecta líneas con la transformada de Hough
min_line_length = canny_masked_frame.shape[0] // 5  # Longitud mínima de la linea
lineas_campo = cv2.HoughLinesP(canny_masked_frame, 1, np.pi / 180, threshold=30, minLineLength=min_line_length, maxLineGap=5)

#cv2.imshow("masked_image", masked_image)

# Filtrar las nlineas líneas más próximas a la parte inferior de la imagen
nlineas = 10
if lineas_campo is not None:
    lineas_campo = sorted(lineas_campo, key=lambda l: min(l[0][1], l[0][3]), reverse=True)
    lineas_campo = lineas_campo[:10]

# Inicializa grupos de líneas y umbral de orientación para separar
group_1 = []  
group_2 = []  
angle_threshold = 10

# Agrupa líneas por orientación
if lineas_campo is not None:
    for line in lineas_campo:
        x1, y1, x2, y2 = line[0]
        
        # Evita división por cero en líneas verticales
        if x2 - x1 == 0:
            angle = 90  
        else:
            # Ángulo en grados
            angle = np.degrees(np.arctan((y2 - y1) / (x2 - x1)))
            angle = abs(angle) 
        
        # Asignar grupo
        if abs(angle) < angle_threshold:  # Cercano a horizontal
            group_1.append(line)
        else:  # Resto
            group_2.append(line)

# Dibuja cada grupo con un color
if group_1:
    for line in group_1:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Azul para grupo 1

if group_2:
    for line in group_2:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Verde para grupo 2

#INTERSECCIONES
# Obtener las intersecciones entre líneas de grupo diferente
image_height, image_width = image.shape[:2]
intersections = []

for i in range(len(group_1)):
    for j in range(len(group_2)):
        line1 = group_1[i][0]
        line2 = group_2[j][0]

        intersection = line_intersection(line1, line2)
        # Chequea que esté en la imagen
        if intersection and is_within_image(intersection, image_width, image_height):
            intersections.append(intersection)

# Dibuja  intersecciones en la imagen
for point in intersections:
    x, y = point
    cv2.circle(image, (int(x), int(y)), 5, (255, 0, 255), -1)

# Muestra imagen con líneas e intersecciones
cv2.imshow("Líneas y esquinas detectadas", image)


cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'masked_frame' is not defined

Segmentación

In [43]:
import cv2
import numpy as np
from ultralytics import YOLO  #Necesario para detectar con YOLO

#Intersección enter dos líneas
def line_intersection(line1, line2):
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2

    # Calcular  pendientes e intersecciones
    denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    if denom == 0:  # Líneas paralelas o coincidentes
        return None

    intersect_x = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denom
    intersect_y = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denom

    return (intersect_x, intersect_y)

#Comprueba que el punto esté dentro de la imagen
def is_within_image(point, image_width, image_height):
    x, y = point
    return 0 <= x < image_width and 0 <= y < image_height


# Cargar el modelo YOLO entrenado
model = YOLO("./yolo_models/segment.pt")

video_path = "./assets/videos/okc-nyk.mp4"
# Cargar el video de entrada
cap = cv2.VideoCapture(video_path)

# Obtener detalles del video
fourcc = cv2.VideoWriter_fourcc(*"avc1")  # Codec para el video de salida
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Procesar cada frame del video
ret, frame = cap.read()
if not ret:
    print("Error al leer el video.")
    cap.release()
    exit()

# Realizar detección en el frame
results = model.predict(frame, task='segment')
# Dimensiones del frame original
frame_height, frame_width = frame.shape[:2]

# Se preestablecen las coordenadas del segmento a investigar
xmin, ymin, xmax, ymax = 0,0,0,0
cropped_zone = 0
# Iterar sobre los resultados
for result in results:
    masks = result.masks.data.cpu().numpy()  # Máscaras (NumPy array)
    classes = result.boxes.cls.cpu().numpy()  # Clases detectadas
    boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes
    image = frame.copy()  # Crear copia del frame original para visualización

    # Aplicar y mostrar máscaras
    for i, cls in enumerate(classes):
        if model.names[cls] == 'three_second_area':
            # Redimensionar la máscara al tamaño del frame original
            zone_mask = (masks[i] * 255).astype(np.uint8)  # Escalar la máscara (0-255)
            resized_zone_mask = cv2.resize(zone_mask, (frame_width, frame_height), interpolation=cv2.INTER_NEAREST)

            # Invertir la máscara
            zone_mask_inv = cv2.bitwise_not(resized_zone_mask)

            # Aplicar la máscara al frame original
            combined = cv2.bitwise_and(image, image, mask=zone_mask_inv)
            
            xmin, ymin, xmax, ymax = boxes[i].astype(int)
            cropped_zone = combined[ymin:ymax, xmin:xmax]

# Mostrar la zona en pantalla
cv2.imshow("La zona", cropped_zone)

# Detección de líneas en la zona designada
image = cropped_zone.copy()
gaussian_image = cv2.GaussianBlur(image, (17,17), 0)



# Forma alternativa
masked_frame_gray = cv2.cvtColor(gaussian_image, cv2.COLOR_BGR2GRAY)

# Forma utilizada
mask = cv2.inRange(gaussian_image, 0,1)
cv2.imshow("mascara", mask)
gaussian_masked_frame = cv2.GaussianBlur(mask, (17,17), 0)
cv2.imshow("gaussian mask", gaussian_masked_frame)
canny_masked_frame = cv2.Canny(gaussian_masked_frame, 30, 150)
cv2.imshow("canny", canny_masked_frame)

# Detecta líneas con la transformada de Hough
min_line_length = canny_masked_frame.shape[0]*0.75  # Longitud mínima de la linea
lineas_campo = cv2.HoughLinesP(canny_masked_frame, rho=1, theta=np.pi / 180, threshold=15, minLineLength=min_line_length, maxLineGap=105)

# Filtrar las nlineas líneas más próximas a la parte inferior de la imagen
nlineas = 10
if lineas_campo is not None:
    lineas_campo = sorted(lineas_campo, key=lambda l: min(l[0][1], l[0][3]), reverse=True)
    lineas_campo = lineas_campo[:10]

# Inicializa grupos de líneas y umbral de orientación para separar
group_1 = []  
group_2 = []  
angle_threshold = 20

# Agrupa líneas por orientación
if lineas_campo is not None:
    for line in lineas_campo:
        print(line)
        x1, y1, x2, y2 = line[0]
        
        # Evita división por cero en líneas verticales
        if x2 - x1 == 0:
            angle = 90  
        else:
            # Ángulo en grados
            angle = np.degrees(np.arctan((y2 - y1) / (x2 - x1)))
            angle = abs(angle) 
        
        # Asignar grupo
        if abs(angle) < angle_threshold:  # Cercano a horizontal
            group_1.append(line)
        else:  # Resto
            group_2.append(line)

# Dibuja cada grupo con un color
if group_1:
    for line in group_1:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Azul para grupo 1

if group_2:
    for line in group_2:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Verde para grupo 2

#INTERSECCIONES
# Obtener las intersecciones entre líneas de grupo diferente
image_height, image_width = image.shape[:2]
intersections = []

for i in range(len(group_1)):
    for j in range(len(group_2)):
        line1 = group_1[i][0]
        line2 = group_2[j][0]

        intersection = line_intersection(line1, line2)
        # Chequea que esté en la imagen
        if intersection:
            intersections.append(intersection)
        # Dibujar el punto    
        x,y = intersection
        if is_within_image(intersection, image_width, image_height):
            cv2.circle(image, (int(x), int(y)), 5, (255, 0, 255), -1)

# "intersections" contiene todos los puntos de intersección de las líneas encontradas en el espacio
print("Puntos de intersección: ")
print(intersections)

# Muestra imagen con líneas e intersecciones
cv2.imshow("Líneas y esquinas detectadas", image)

# Esperar a que se presione una tecla y cerrar ventana
cv2.waitKey(0)

cv2.destroyAllWindows()


0: 384x640 1 backboard, 1 backboard_inner_square, 1 court, 1 hoop, 1 net, 1 three_second_area, 1 two_point_area, 45.6ms
Speed: 2.0ms preprocess, 45.6ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)
[[132 143 258 128]]
[[210 133 332 124]]
[[137 143 518  97]]
[[234 132 575  91]]
[[  9  64 127 143]]
[[213  29 344  15]]
[[220  27 346  14]]
[[416  13 576  81]]
[[  9  53 414  10]]
[[  9  52 410  10]]
Puntos de intersección: 
[(np.float64(127.75486182190379), np.float64(143.5053735926305)), (np.float64(592.8052516411378), np.float64(88.1422319474836)), (np.float64(121.78373831775701), np.float64(139.5077570093458)), (np.float64(626.1232539030402), np.float64(102.30238290879211)), (np.float64(128.5278520561826), np.float64(144.0228840037155)), (np.float64(592.4867139593604), np.float64(88.00685343272815)), (np.float64(129.36170815369607), np.float64(144.58114359442365)), (np.float64(594.1202097620009), np.float64(88.70108914885034)), (np.float64(-8.000416631947338), np.floa